In [1]:
%load_ext sql
%sql sqlite:///chinook.db

## Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (судить по количеству купленных треков).

In [2]:
%%sql

SELECT c.Country, count(ii.InvoiceLineId) as sold_count FROM Customers c

JOIN Invoices       i   ON c.CustomerId = i.CustomerId
JOIN Invoice_items  ii  ON i.InvoiceId  = ii.InvoiceId
JOIN Tracks         t   ON ii.TrackId   = t.TrackId
JOIN Albums         al  ON t.AlbumId    = al.AlbumId
JOIN Artists        ar  ON al.ArtistId  = ar.ArtistId

WHERE ar.Name = "Iron Maiden"

GROUP BY c.Country

ORDER BY sold_count DESC

LIMIT 1;

 * sqlite:///chinook.db
Done.


Country,sold_count
USA,34


## Задание 2
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом *(цена альбома - сумма цен треков)*.

In [3]:
%%sql

WITH AlbumPrices as 
(
    SELECT al.AlbumId, sum(t.UnitPrice) AS TotalPrice FROM Albums al

    JOIN Tracks t ON al.AlbumId = t.AlbumId

    GROUP BY al.AlbumId
),

AveragePrice as 
(
    SELECT avg(TotalPrice) AS AvgPrice FROM AlbumPrices
)

SELECT al.Title, sum(t.Milliseconds) as Duration FROM Albums al

JOIN Tracks         t   ON al.AlbumId = t.AlbumId
JOIN AlbumPrices    ap  ON al.AlbumId = ap.AlbumId

CROSS JOIN AveragePrice

WHERE ap.TotalPrice > AveragePrice.AvgPrice

GROUP BY al.AlbumId

ORDER BY Duration DESC

LIMIT 1;

 * sqlite:///chinook.db
Done.


Title,Duration
"Lost, Season 3",70665582


## Задание 3
Вывести общую стоимость треков, приобретённых компаниями (только клиентами из конкретных компаний), которые обслуживались сотрудниками, нанятыми после 3 августа 2002 года.

In [4]:
%%sql

SELECT sum(ii.UnitPrice * ii.Quantity) AS TotalSales FROM Invoice_items ii

JOIN Invoices   i ON ii.InvoiceId   = i.InvoiceId
JOIN Customers  c ON i.CustomerId   = c.CustomerId
JOIN Employees  e ON c.SupportRepId = e.EmployeeId

WHERE e.HireDate > '2002-08-03';

 * sqlite:///chinook.db
Done.


TotalSales
1495.5600000000115


## Задание 4
Для каждого альбома определить все жанры песен, которые в нём есть и количество песен по каждому жанру (формат вывода: альбом, жанр, количество).
Выводить только те альбомы, где жанров больше одного.

In [5]:
%%sql

WITH AlbumGenres as 
(
    SELECT al.AlbumId, al.Title AS Album, g.Name AS GenreName, count(t.TrackId) as TrackCount FROM Albums al

    JOIN tracks t ON al.AlbumId = t.AlbumId
    JOIN genres g ON t.GenreId  = g.GenreId

    GROUP BY al.AlbumId, g.GenreId
)

SELECT ag.Album, ag.GenreName, ag.TrackCount FROM AlbumGenres ag

JOIN 
(
    SELECT AlbumId FROM AlbumGenres

    GROUP BY AlbumId

    having count(DISTINCT GenreName) > 1
) 
FilteredAlbums ON ag.AlbumId = FilteredAlbums.AlbumId

ORDER BY ag.AlbumId, ag.GenreName;

 * sqlite:///chinook.db
Done.


Album,GenreName,TrackCount
Unplugged,Blues,14
Unplugged,Latin,16
Live After Death,Heavy Metal,7
Live After Death,Metal,11
Rock In Rio [CD2],Metal,1
Rock In Rio [CD2],Rock,8
The Number of The Beast,Metal,7
The Number of The Beast,Rock,1
Greatest Hits,Metal,14
Greatest Hits,Reggae,13


## Задание 5
Для каждого артиста (таблицы Artists), у которых есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение суммы, заработанной артистом по этому жанру к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, максимальное для артиста, то выбирается жанр раньше по алфавиту. Формат вывода: артист, жанр, соотношение.

In [6]:
%%sql

WITH ArtistGenreSales as
(
    SELECT ar.ArtistId, ar.Name as ArtistName, g.Name as GenreName, sum(t.UnitPrice * ii.Quantity) as GenreSales
    FROM Artists ar

    JOIN Albums         al  ON ar.ArtistId  = al.ArtistId
    JOIN Tracks         t   ON al.AlbumId   = t.AlbumId
    JOIN Genres         g   ON t.GenreId    = g.GenreId
    JOIN Invoice_items  ii  ON t.TrackId    = ii.TrackId

    GROUP BY ar.ArtistId, g.Name
),

Total as 
(
    SELECT ArtistId, sum(GenreSales) as TotalSales FROM ArtistGenreSales

    GROUP BY ArtistId
)

SELECT ags.ArtistName, ags.GenreName, round(ags.GenreSales / Total.TotalSales, 3) as SalesRatio FROM ArtistGenreSales ags

JOIN Total ON ags.ArtistId = Total.ArtistId

WHERE ags.GenreSales = (
        SELECT max(ags_inn.GenreSales) FROM ArtistGenreSales ags_inn
        WHERE ags_inn.ArtistId = ags.ArtistId
    )
    AND 
    ags.GenreName = (
        SELECT min(ags_inn.GenreName) FROM ArtistGenreSales ags_inn
        WHERE ags_inn.ArtistId = ags.ArtistId AND ags_inn.GenreSales = ags.GenreSales
    )

GROUP BY ags.ArtistId

ORDER BY ags.ArtistName;

 * sqlite:///chinook.db
Done.


ArtistName,GenreName,SalesRatio
AC/DC,Rock,1.0
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,1.0
"Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair",Classical,1.0
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,1.0
Accept,Rock,1.0
Adrian Leaper & Doreen de Feis,Classical,1.0
Aerosmith,Rock,1.0
Alanis Morissette,Rock,1.0
Alice In Chains,Rock,1.0
Amy Winehouse,R&B/Soul,0.667
